## load libs

In [1]:
import cpufeature as cpufeature
import dcimg as dcimg
import numba as numba
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from process_images import *
from pystripe.core import *
import matplotlib.pyplot as plt
def plot_images(img_list: List[ndarray], img_labels: List[str], vmax: int):
    fig, axes = plt.subplots(nrows=1, ncols=len(img_list), figsize=(20, 20))
    for idx, (im, label) in enumerate(zip(img_list, img_labels)):
        axes[idx].imshow(im, cmap='gray', vmin=0, vmax=vmax)
        axes[idx].set_title(label)
    plt.tight_layout()
    plt.show()

In [ ]:
from parallel_image_processor import *
tsv_volume = TSVVolume.load(r'E:\20230510_13_34_13_SM230308_05_LS_15x_800z_MIP_stitched\Ex_488_Em_525_MIP_xml_import_step_5.xml')
shape: Tuple[int, int, int] = tsv_volume.volume.shape  # shape is in z y x format
img = tsv_volume.imread(
    VExtent(
        tsv_volume.volume.x0, tsv_volume.volume.x1,
        tsv_volume.volume.y0, tsv_volume.volume.y1,
        tsv_volume.volume.z0 + shape[0]//2, tsv_volume.volume.z0 + shape[0]//2 + 1),
    tsv_volume.dtype)[0]
parallel_image_processor(
    source=TSVVolume.load(r'/data/20230419_17_34_03_SM221011_06_LS_15x_800z_stitched/Ex_488_Em_525_xml_import_step_5.xml'),
    destination=r"/data/20230419_17_34_03_SM221011_06_LS_15x_800z_stitched/Ex_488_Em_525_tif",
    fun=process_img,
    kwargs={'bleach_correction_frequency': 0.0005, 'bleach_correction_max_method': False, 'bleach_correction_y_slice_max': None, 'threshold': None, 'sigma': (4000.0, 4000.0), 'bidirectional': True, 'lightsheet': False, 'percentile': 0.25, 'rotate': 90, 'convert_to_8bit': False, 'bit_shift_to_right': 8, 'tile_size': (39220, 28056), 'd_type': 'uint16', "verbose": True},
    source_voxel=(0.8, 0.4, 0.4),
    target_voxel=20,
    max_processors=1
)

In [15]:
import cv2
import numpy as np
from filestack import FileStack

# 0 < percentile < 100
def get_matrix_two_images(image1, image2, percentile):
    # there should never be a case where both images are none, but this function checks just in case.
    if image1 is None and image2 is None: return None

    # check image1 type
    if not isinstance(image1, ndarray) and not image1 is None:
        #normally we need to convert the .tif file to be gray, but these tif files are already single-channel.
        img1 = cv2.imread(image1, flags=cv2.IMREAD_ANYDEPTH)
    elif image1 is None: return image2
    else: img1 = image1

    # check image2 type
    if not isinstance(image2, ndarray) and not image2 is None:
        img2 = cv2.imread(image2, flags=cv2.IMREAD_ANYDEPTH)
    elif image2 is None: return image1
    else: img2 = image2

    # generate matrices
    img1_percentile = np.percentile(img1, percentile)
    img2_percentile = np.percentile(img2, percentile)

    # scale images to max brightness at percentile; 0 <= pixel intensity <= 1
    for i in range(len(img1)):
        for j in range(len(img1[0])):
            if img1[i][j] > img1_percentile:
                img1[i][j] = img1_percentile
            img1[i][j] *= (1 / img1_percentile)
    for i in range(len(img2)):
        for j in range(len(img2[0])):
            if img2[i][j] > img2_percentile:
                img2[i][j] = img2_percentile
            img2[i][j] *= (1 / img2_percentile)

    warp_mode = cv2.MOTION_TRANSLATION
    warp_matrix = np.eye(2, 3, dtype=np.float32)
    num_iter = 10000
    termination_eps = 1e-10
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, num_iter, termination_eps)

    cc, warp_matrix = cv2.findTransformECC(img1, img2, warp_matrix, warp_mode, criteria)
    return warp_matrix

def generate_combined_image(images, matrices):
    # there should always be 1 more image than number of matrices.  Example: 3 images, 2 matrices.
    if len(images) != len(matrices) + 1:
        raise Exception("Images and matrices arrays have incorrect lengths")
    img0 = cv2.imread(str(images[0].absolute()), flags=cv2.IMREAD_ANYDEPTH)
    size = img0.shape
    transformed_images = [img0]
    for i in range(1, len(images)):
        if not images[i] is None:
            read_img = cv2.imread(str(images[i].absolute()), flags=cv2.IMREAD_ANYDEPTH)
        else:
            read_img = np.zeros(size, dtype = img0.dtype)

        warped_image = cv2.warpAffine(read_img, matrices[i - 1], (size[1], size[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        transformed_images.append(warped_image)


    merged = cv2.merge(transformed_images)

    # for testing purposes
    # cv2.imshow("Image 0", transformed_images[0])
    # cv2.imshow("Image 1", transformed_images[1])
    # cv2.imshow("Image 2", transformed_images[2])
    # cv2.imshow("Merged", merged)
    # cv2.waitKey(0)
    return merged

def merge_channels(paths, output):
    print("Merging")
    stacks = []
    for i in paths:
        stacks.append(FileStack(i))
    largest_stack_index = max([i.get_count() for i in stacks])

    # generates a matrix using middle images of each stack, then apply it to all images in the stacks
    # should not be parallelized; this is only done once.
    ind = list(map(lambda x: x.get_count() // 2, stacks))
    percentile = 90
    matrices = list(map(lambda p: get_matrix_two_images(str(stacks[0].get_index(ind[0]).absolute()), p, percentile), [str(stacks[i].get_index(ind[i]).absolute()) for i in range(1, len(stacks))]))

    # apply transformations to every image
    # loop can be replaced with stack and multiple threads
    for i in range(largest_stack_index):
        filename = str(output.absolute() / 'merged_') + str(i) + ".tif"
        imsave_tif(filename, generate_combined_image([j.get_index(i) for j in stacks], matrices))


img_paths = [
    Path("./tempData/cha1"),
    Path("./tempData/cha2"),
    Path("./tempData/cha3")
]
output_path = Path("./tempData/merged")
merge_channels(img_paths, output_path)

Merging
